In [ ]:
from google.colab import files
from google.colab import drive
import json
import itertools
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import joblib
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
drive.mount('/content/drive')
%cd drive/MyDrive/HackUPC

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/HackUPC'
/content/drive/MyDrive/HackUPC


In [ ]:
df = pd.read_csv("reduced.csv")

In [ ]:
print(df.columns)

Index(['city', 'neighborhood', 'region', 'price', 'square_meters', 'bedrooms',
       'bathrooms', 'property_rating', 'kitchen_rating', 'bathroom_rating',
       'interior_rating'],
      dtype='object')


In [ ]:
print(df['neighborhood'].value_counts())

Dénia                    1921
Vitoria - Gasteiz        1166
Torrevieja                947
Calp                      839
Ferrol                    754
                         ... 
Bisaurri                    1
Breña Alta                  1
Marina de Cudeyo            1
San Felices de Buelna       1
Sant Joan d'Alacant         1
Name: neighborhood, Length: 907, dtype: int64


In [ ]:
# Definir los arrays de códigos postales
denia_zipcodes = [39586, 39216, 39267, 39211]
vitoria_zipcodes = [26386, 26216, 26267, 26211]
torrevieja_zipcodes = [17386, 17216, 17267, 17211]

# Definir la función para asignar el código postal según el barrio
def assign_zipcode(row):
    if row['neighborhood'] == 'Dénia':
        return np.random.choice(denia_zipcodes)
    elif row['neighborhood'] == 'Vitoria - Gasteiz':
        return np.random.choice(vitoria_zipcodes)
    elif row['neighborhood'] == 'Torrevieja':
        return np.random.choice(torrevieja_zipcodes)
    else:
        return np.nan

# Aplicar la función a cada fila del dataframe
df['zipcode'] = df.apply(assign_zipcode, axis=1)

In [ ]:
df.head()

,city,neighborhood,region,price,square_meters,bedrooms,bathrooms,property_rating,kitchen_rating,bathroom_rating,interior_rating,zipcode
0,Ames,San Tome,a_coruna,250000,315,4,3,4.2,4.7,4.0,4.0,NaN
1,Ames,San Tome,a_coruna,90000,72,2,1,3.5,3.4,3.5,3.7,NaN
2,Perillo,Oleiros,a_coruna,359000,244,4,3,3.2,3.1,3.6,3.0,NaN
3,Catro Camiños-A Gaiteira,A Coruña Capital,a_coruna,102000,48,1,1,4.2,4.5,4.5,3.8,NaN
4,Bergondo,Carrio,a_coruna,285000,300,4,4,3.1,3.4,3.1,2.8,NaN


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

,city,neighborhood,region,price,square_meters,bedrooms,bathrooms,property_rating,kitchen_rating,bathroom_rating,interior_rating,zipcode
415,Lakua-Arriaga,Vitoria - Gasteiz,alava_araba,184000,2,2,2,3.5,3.8,3.8,3.0,26211.0
417,Arana-El Anglo-Santiago,Vitoria - Gasteiz,alava_araba,299000,90,3,2,3.7,3.6,3.5,4.0,26211.0
418,Ali-Gobeo,Vitoria - Gasteiz,alava_araba,95000,3,3,1,4.3,4.5,4.7,3.8,26211.0
419,Arana-El Anglo-Santiago,Vitoria - Gasteiz,alava_araba,180000,2,2,1,3.5,3.6,3.4,3.4,26211.0
420,Zabalgana,Vitoria - Gasteiz,alava_araba,158000,88,3,2,3.5,3.3,4.1,3.2,26216.0


In [ ]:
df.shape[0]

4034

In [ ]:
df.drop('city', axis=1, inplace=True)
df.drop('neighborhood', axis=1, inplace=True)
df.drop('region', axis=1, inplace=True)

In [ ]:
train_df = df.copy()
X = train_df.drop('price', axis=1)
y = train_df['price']

In [ ]:
# crear el modelo
rf = RandomForestRegressor(n_estimators=420, max_depth=69, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=69, n_estimators=420, random_state=42)

In [ ]:
print(X_test.columns)
print(X_test.shape)

Index(['square_meters', 'bedrooms', 'bathrooms', 'property_rating',
       'kitchen_rating', 'bathroom_rating', 'interior_rating', 'zipcode'],
      dtype='object')
(807, 8)


In [ ]:
print(X_test[:5])

       square_meters  bedrooms  bathrooms  property_rating  kitchen_rating  \
32754             90         3          2              4.7             4.8   
570              122         3          2              4.6             4.8   
29711            125         3          2              4.0             3.9   
29955              2         2          1              4.1             4.2   
29784             57         1          1              3.7             3.7   

       bathroom_rating  interior_rating  zipcode  
32754              4.8              4.5  17211.0  
570                4.1              4.8  26386.0  
29711              4.3              3.9  39267.0  
29955              4.5              3.7  39267.0  
29784              3.8              3.7  39211.0  


In [ ]:
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R²:", r2)

R²: 0.8049939722434397


In [ ]:
#joblib.dump(rf, "./random_forest_v2.joblib")